<a href="https://colab.research.google.com/github/tamburins/ESAA_2023/blob/main/ESAA_4_10(487_496).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 04 텍스트 분류 실습 - 20 뉴스그룹 분류

사이킷런 내부 예제데이터를 활용하여 텍스트분류를 적용해보고자 한다. 이는 학습데이터를 통해 모델을 생성한 뒤 학습모델을 이용해 다른 문서의 분류를 예측하는 것이다.

사이킷런은 fetch_20newsgroupas() api를 이용하여 예제데이터를 제고하며 텍스트를 피처벡터화로 변환하면 일반적으로 희소행렬 형태가 되며 이렇나 희소행렬에 분류를 효과적으로 잘 처리할 수 있는 알고리즘은 로지스틱회귀, 선형서포트 벡터머신, 나이브베이즈 등이다. 그 중 로지스틱 회귈르 이용해 분류를 수행하고자 한다. 텍스트를 기반으로 분류를 수행할 때에는텍스틀르 정규화한 뒤 피처 벡터화를 이용하고 ml 알고리즘을 적용해 분류를 학습 예측 평가한다. 이번 절에서는 카운트 기반과 tf idf 기반으 ㅣ벡터화를 차례로 적용해 예측 성능을 비교하고 피처 벡터화를 위한 파라미터 grid search cv기반의 하이퍼 파라미터 기반의 하이퍼 파라미터 튜닝을 한꺼번에 하고자 한다.

### 텍스트 정규화



In [ ]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset = 'all', random_state = 156)

print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


load_xxx와 유사한 키값을 가진다. 다음으로 타겟클래스 구성이다.

In [ ]:
import pandas as pd
print('target 클래스의 값과 분포도 \n', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


타겟 클래스의 값은 0부터 19까지 20개로 구성되며 위의 출력결과처럼 주어졌다. 그 중 개별데이터가 텍스트로 어떻게구성되었는지 하나만 추출해알아보고자 한다.

In [ ]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

단순 뉴스뿐만 아니라 뉴스그룹 제목 작성자 소속 이메일 등 다양한 정보를 가지며 이 중 내용을 제거하고 다른 정보는 제거한다. 이는 target 클래스값과 다들 비슷하여 고성능의 예측 알고리즘이 되기 때문이다. remove파라미터를 이용하여 뉴스그룹 기사의 헤더, 푸터 등을 제거할 수 ㅣㅇㅆ다. 또한 fetch_20newsgroup는 subset 파라미터를 이용해 학습 데이터 세트와 테스트 데이터 세트를 분리해 내려받을 수 있다.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
# subset = 'train'으로 학습용 데이터만 추출, remove=('headers', 'footers', 'quotes') 로 내용만 추출
train_news = fetch_20newsgroups(subset = 'train', remove=('headers', 'footers', 'quotes'), random_state = 156)

X_train = train_news.data
y_train = train_news.target

#subset='test'으로 테스트 데이터만 추출, remove=('headers', 'footers', 'quotes') 로 내용만 추출

test_news = fetch_20newsgroups(subset = 'train', remove=('headers', 'footers', 'quotes'), random_state = 156)

X_test = train_news.data
y_test = train_news.target
print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data),len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 11314


### 피처 벡터화 변환과 머신러닝 모델 학습 예측 평가

countvectorizer를 이용해 학습데이터의 텍스트를 피처 벡터화 하고자 한다. 테스트 데이터 또한 피처벡터화를 수행하는데, 한가지 유의할 것은 테스트 데이터에서 countvectorizer를 적용할 때에는 학습데이터를 이용해 fit 수행된 카운트벡터라이즈 객체를 이용해 테스트데이터를 변환해야한다는 것이다.

테스트데이터의 피처벡터화시 fit_transform을 사용하면 안된다.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

#Count Vectorization으로 피처 벡터화 변환 수행.
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

#학습 데이터로 fit()된 CountVectorizer를 이용해 체스트 데이터를 피터 벡처화 변환 수행.
X_test_cnt_vect= cnt_vect.transform(X_test)

print('학습 데이터 텍스트의 countVectorizer shape:', X_train_cnt_vect.shape)
     

학습 데이터 텍스트의 countVectorizer shape: (11314, 101631)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#LogisticRegression을 이용해 학습/예측/평가 수행.
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred= lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))
     

CountVectorized Logistic Regression의 예측 정확도는 0.964


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환.
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

#LogisticRegression을 이용해 학습/예측/평가 수행

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred= lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test, pred)))
     

TF-IDF Logistic Regression의 예측 정확도는 0.907


tf idf 가 단순 카운트기반보다 높은 예측 정확도를 제공하는데 일반적으로 문서 내에 텍스트가 많고 많은 문서를 갖는 텍스트 분석에서 카운트 벡터화보다는 tfidf 벡터화가 좋은 예측결과를 도출한다.

텍스트분석에서 머신러닝 모델의 성능을 향상시키는 중요한 방법은 최적의 ml 모델을 선택하는 것과 최상의 피처 전처리를 하는 것인데 이번에는 다양한 파라미터를 적용해보겠다.

In [ ]:
# stop words 필터링을 추가하고 ngram을 기본 (1,1)에서 (1,2)로 변경해 피처 벡터화 적용
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidt_vect = tfidf_vect.transform(X_train)
X_test_tfidt_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidt_vect, y_train)
pred = lr_clf.predict(X_test_tfidt_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:3f}'.format(accuracy_score(y_test, pred)))
     

TF-IDF Logistic Regression의 예측 정확도는 0.956161


In [ ]:
from sklearn.model_selection import GridSearchCV

# 최적 C값 도출 튜닝 수행. CV는 3폴드 세트로 설정
params = { 'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(X_train_tfidt_vect, y_train)
print('Logistic Regression best C parameter :', grid_cv_lr.best_params_)
     

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

Logistic Regression best C parameter : {'C': 10}


로지스틱 회귀의 c가 10일 때 gridsearch cv 굧, 검증 테스트세트에서 가장 좋은 예측 성능을 나타냈으며 이를 테스트 데이터 세트에 적용해 이전보다 향상된 성능수치가 되었다.

### 사이킷런 파이프라인 사용 및 gridserachcv와의 결합

사이킷런의 파이프라인 클래스를 이용하면 피처벡터화와 ml 알고리즘 학습 예측을 위한 코드 작성을 한번에 진행할 수 있다. 일반적으로 머신러닝에서 파이프라인이란 데이터의 가공 변환 등으 ㅣ전처리와 알고리즘 적용을 한꺼번에 스트림 기반으로 처리한다는 의미로 통일된 api 기반에서 처리할 수 있으며 대용량 데이터의 피처벡터화 결과를 별도데이터로 저장없이 스트림 기반에서 바로 머신러닝 알고리즘 데이터로 입력할 수 있기 때문에 수행시간을 절약할 수 있다.

일반적으로 사이킷런 파이프라인은 텍스트기반의 피처벡터화 분만 아니라 모든 데이터 전처리 작업과 estimator를 결합할 수 있다. 

In [ ]:
from sklearn.pipeline import Pipeline

# TfidVectorizer 객체를 tfid_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline 생성
pipeline=Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
    ('lr_clf', LogisticRegression(C=10))
])

# 별도의 TfidVectorizer 객체의 fit(), transform()과 LogisticRegression의 fit(), predict()가 필요없음
# pipeline의 fit과 predict()만으로 한꺼번에 피처 벡터화와 ML 학습/예측이 가능
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:3f}'.format(accuracy_score(y_test, pred)))
     

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline을 통한 Logistic Regression의 예측 정확도는 0.972865


위의 코드는 tfidf vectorizer > tfidf, logisticregeression> lr_clf로 각각의 객체를 객체변수명으로 저장한 뒤 이 두개의 객체를 파이프라인으로 연결하는 pipeline을 생성한다는 의미이다.

또한 기존 tfidfvectorizer의 학습데이터와 테스트데이터에 대한 fit()과 transform()수행을 통한 피처벡터화와 로지스틱 리그레서의 fit과 predict() 수행을 통한 머신러닝 모델의 학습과 예측이 파이프라인의 핏과 프레딕트로 통일돼 수행됨을 알 수 있다.


사이킷런은 그리드서치시비 클래스의 생성 파라미터로 파이프라인을 입력해 파이프라인 기밪ㄴ에서도 하이퍼 파라미터 튜닝을 그리드서치시비 방식으로 진행할 수 있게 지원된다. 이를 통해 피처벡터화를 위한 파라미터와 ml 알고리즘의 하이퍼파라미터를 한꺼번에 최적화할 수 있다.

다음은 gridserachcv에 파이프라인을 입력하면서 tfidfvectorizer의 파라미터와 logisticregression의 하이퍼파라미터를 함게 최적화한다.예제 코드를 보면 gridserachcv에 estimator가 아닌 pipeline을 입력할 경우 param grid의 입력값 설정이 기존과 약간 다르다. 딕셔너리 형태의 키와 value값을 가지며 value값이 리스트 형태로 입력되는 것은 동일하나 하이퍼파라미터명과 객체변수명이 합쳐쳐저서 제공된다. 파이프라인을 그리드서치에 인자로 입력하면 그리드서치는 파이프라인을 구성하는 피처벡터와 객체의 파라미터 estimator객체 셋의 하이퍼 파라미터를 모두 구별할 수 있어야 하는데, 이 때 개별 객체명과 파라미터명/하이퍼파라미터명을 결합해 키값으로 할당하는 것이다. 가령 tfidfvectorizer 객체변수인 tfidf_vect에 ngram_range 파라미터 값을 변화시키며 최적화하기를 원한다면 객체변수명인 tfidf_Vect에 언더바를 붙여 ngram_range를 결합해 tfidf_vect__ngram_range를 키로 할당하는 것이다.

유의점은 모두 최적화하려면 튜닝시간이 많이 소요된다는 것이다.

In [ ]:
from sklearn.model_selection import GridSearchCV

pipeline=Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])

# Pipeline에 기술된 각각의 객체 변수에 언더바(__) 2개를 연달아 붙여 GridSearchCV에 사용될 파라미터/하이퍼파라미터 이름과 값을 설정
params = {'tfidf_vect__ngram_range':[(1,1),(1,2),(1,3)],
         'tfidf_vect__max_df': [100,300,700],
         'lr_clf__C':[1,5,10]}

# GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}.format(accuracy_score(y_test,pred))')
     


Fitting 3 folds for each of 27 candidates, totalling 81 fits


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

{'lr_clf__C': 10, 'tfidf_vect__max_df': 300, 'tfidf_vect__ngram_range': (1, 2)} 0.7536687914006531
Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}.format(accuracy_score(y_test,pred))


tfidf vectorizer 객체의 max_df 파라미터가 700,ngram range 파라미터가 1,2로 피처벡터화된 데이터세터에 logisticregression c 하이퍼파라미터에 10을 적용해 예측분류를 수행할 때 가장 좋은 검증세트 수치가 도출되었으나 정확도가 크개 개선되지 않았다.